In [1]:
import sys
sys.path.append("/Users/nicolasaunai/Documents/code/phare/phare/pharein")
import pharein as ph

In [2]:
ph.globals.sim = None

# user initialization script

In [3]:
#!/usr/bin/env python

import pharein
from pharein import Simulation
from pharein import MaxwellianFluidModel
from pharein import ElectromagDiagnostics
from pharein import FluidDiagnostics
from pharein import ParticleDiagnostics

In [4]:

#------------------------------------
#     configure the simulation
#------------------------------------

Simulation(
    time_step_nbr=1000,                   # number of time steps (not specified if time_step and final_time provided)
    final_time=1.,                        # simulation final time (not specified if time_step and time_step_nbr given)
    boundary_types="periodic",            # boundary condition, string or tuple, length == len(cell) == len(dl)
    cells=80,                             # integer or tuple length == dimension
    dl=0.1,                               # mesh size of the root level, float or tuple
    path='test5'                          # directory where INI file and diagnostics directories will be
    # time_step = 0.005,                  # simulation time step (not specified if time_step_nbr and final_time given)
    # domain_size = 8.,                   # float or tuple, not specified if dl and cells are
    # interp_order = 1,                   # interpolation order, [default = 1] can be 1, 2, 3 or 4
    # layout = "yee",                     # grid layout, [default="yee"]
    # origin = 0.,                        # position of the origin of the domain, float or tuple (length = dimension)
    # particle_pusher = "modified_boris", # particle pusher method, [default = "modified_boris"]
    # refined_particle_nbr = 2,           # number of refined particle a particle is split into [default : ]
    # diag_export_format = 'ascii',       # export format of the diagnostics [default = 'ascii']
    # refinement = {"level":[0,1],        # AMR parameters
    #                "extent_ratio":[0.4, 0.6],
    #                "refinement_iterations":[0, 3]},

) # end Simulation





Need to test boxes are OK (no overlap, in level range, etc.)


In [5]:

import numpy as np

def n(x):
    x0 = 5.
    return 1./np.cosh(x-x0)**2

def bx(x):
    x0=5.
    return np.tanh(x-x0)


MaxwellianFluidModel(bx=bx,
                     protons={"density":n},
                     background={})





In [6]:
ElectromagDiagnostics(
    diag_type="E",                  # available : ("E", "B")
    write_every=10,
    compute_every=5,
    start_teration=0,
    last_iteration=990,
    path = 'ElectromagDiagnostics1'   # where output files will be written, [default: name]
) 

In [7]:
FluidDiagnostics(
    diag_type="density",            # choose in (rho_s, flux_s)
    write_every=10,                 # write on disk every x iterations
    compute_every=5,                # compute diagnostics every x iterations ( x <= write_every)
    start_iteration=0,              # iteration at which diag is enabled
    last_iteration=990,             # iteration at which diag is turned off
    population_name="protons"       # name of the population for which the diagnostics is made
  #,path = 'FluidDiagnostics1'      # where output files will be written, [default: name]
)

In [8]:
FluidDiagnostics(
    diag_type="bulk_velocity",
    write_every=10,
    compute_every=5,
    start_iteration=0,
    last_iteration=990,
    population_name="background"
)

In [9]:
FluidDiagnostics(
    diag_type="density",
    write_every=10,
    compute_every=5,
    start_iteration=0,
    last_iteration=990,
    population_name="all"
)

In [10]:
pharein.globals.sim.diagnostics

In [11]:
FluidDiagnostics(
    diag_type="flux",
    write_every=10,
    compute_every=5,
    start_iteration=0,
    last_iteration=990,
    population_name="background"
)

In [12]:
ElectromagDiagnostics(
    diag_type="B",
    write_every=10,
    compute_every=5,
    start_teration=0,
    last_iteration=990
)

In [13]:
ParticleDiagnostics(
        compute_every=10,
        write_every=10,
        start_iteration=0,
        last_iteration=90,
        diag_type="space_box",          # choose particles within a spatial box
        extent=(2., 4.),                # extent of the box
        population_name="protons"
        )

In [14]:
[diag.name for diag in ph.globals.sim.diagnostics]

['electromag0',
 'fluid0',
 'fluid1',
 'fluid2',
 'fluid3',
 'electromag1',
 'particle0']

In [15]:
ph.globals.sim.model.model_dict

{'model': 'model',
 'model_name': 'custom',
 'bx': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
 'by': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
 'bz': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
 'ex': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
 'ey': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
 'ez': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
 'protons': {'charge': 1.0,
  'mass': 1.0,
  'density': <function __main__.n(x)>,
  'vx': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
  'vy': <function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>,
  'vz': <function pharein.maxwellian_fluid_model.Maxwell

In [16]:
d = ph.globals.sim.model.model_dict["protons"]["density"]

In [17]:
d(2)

0.009866037165440192

In [18]:
ph.globals.sim.model.model_dict['bx']

<function pharein.maxwellian_fluid_model.MaxwellianFluidModel.defaulter.<locals>.<lambda>(x)>

In [19]:
for diag in ph.globals.sim.diagnostics:
    diag_path = "simulation/diagnostics/"
    categ_path = diag_path + diag.category + '/'
    name_path = categ_path + diag.name
    print(name_path + "/" + 'subtype/' + diag.diag_type)
    print(name_path + "/" + 'compute_every/' + str(diag.compute_every))
    print(name_path + "/" + 'write_every/' + str(diag.write_every))
    print(name_path + "/" + 'start_iteration/' + str(diag.start_iteration))
    print(name_path + "/" + 'last_iteration/' + str(diag.last_iteration))    
    if (diag.category in ["FluidDiagnostics", "ParticleDiagnostics"]):
        print(name_path + "/" + 'population/' + diag.population_name)    
        
    
    #print("simulation/diagnostics/" + diag.category)
    #print("simulation/diagnostics/name/" + diag.name)
    #print("simulation/diagnostics/type/"  + diag.category)
    #print("simulation/diagnostics/subtype/"  + diag.diag_type)
    #print("simulation/diagnostics/compute_every/" + str(diag.compute_every))
    #print("simulation/diagnostics/write_every/" + str(diag.write_every))
    #print("simulation/diagnostics/start_iteration/" + str(diag.start_iteration))    
    #print("simulation/diagnostics/last_iteration/" + str(diag.last_iteration))  

    #    print("simulation/diagnostics/population/" + str(diag.population_name))
    print("")
    

simulation/diagnostics/electromag/electromag0/subtype/EM_E
simulation/diagnostics/electromag/electromag0/compute_every/5
simulation/diagnostics/electromag/electromag0/write_every/10
simulation/diagnostics/electromag/electromag0/start_iteration/0
simulation/diagnostics/electromag/electromag0/last_iteration/990

simulation/diagnostics/fluid/fluid0/subtype/density
simulation/diagnostics/fluid/fluid0/compute_every/5
simulation/diagnostics/fluid/fluid0/write_every/10
simulation/diagnostics/fluid/fluid0/start_iteration/0
simulation/diagnostics/fluid/fluid0/last_iteration/990

simulation/diagnostics/fluid/fluid1/subtype/bulk_velocity
simulation/diagnostics/fluid/fluid1/compute_every/5
simulation/diagnostics/fluid/fluid1/write_every/10
simulation/diagnostics/fluid/fluid1/start_iteration/0
simulation/diagnostics/fluid/fluid1/last_iteration/990

simulation/diagnostics/fluid/fluid2/subtype/density
simulation/diagnostics/fluid/fluid2/compute_every/5
simulation/diagnostics/fluid/fluid2/write_every/